In [12]:
from collections import defaultdict
from pathlib import Path

import gurobipy as gp
import polars as pl
from gurobipy import GRB


In [13]:
discount_factor = 5
data_dir = "C:/MyTemp/data/two_stage_data/h1h23"
df = pl.read_csv(Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000)
df = df.select(
    [
        "unit",
        "schedule",
        f"npv_{discount_factor}_percent",
        "stock_0",
        "stock_5",
        "stock_25",
        "harvest_value_5",
        "harvest_value_25",
        "stock_1_0",
        "stock_1_5",
        "stock_1_25",
        "stock_2_0",
        "stock_2_5",
        "stock_2_25",
        "stock_30_0",
        "stock_30_5",
        "stock_30_25",
    ]
)
df_keys = pl.read_csv(Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000)
df_keys = df_keys.drop("holding")
# Split the treatments into multiple columns, to make them easier to read or whatever
df_keys = df_keys.with_columns([
    pl.when(pl.col("treatment").str.contains("_5"))
      .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
      .otherwise(pl.lit("donothing"))
      .alias("treatment_5"),

    pl.when(pl.col("treatment").str.contains("_25"))
      .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
      .otherwise(pl.lit("donothing"))
      .alias("treatment_25")
])

df = df_keys.join(df, on=["unit", "schedule"], how="inner")

In [14]:
# Increase display settings
#pl.Config.set_tbl_rows(1000)    # show up to 1000 rows
#pl.Config.set_tbl_cols(50)      # show up to 50 columns
#pl.Config.set_fmt_str_lengths(100)  # allow long string truncation length

# Now printing will show more
df

unit,schedule,treatment,treatment_5,treatment_25,npv_5_percent,stock_0,stock_5,stock_25,harvest_value_5,harvest_value_25,stock_1_0,stock_1_5,stock_1_25,stock_2_0,stock_2_5,stock_2_25,stock_30_0,stock_30_5,stock_30_25
i64,i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
43124559,0,"""donothing""","""donothing""","""donothing""",5342.485761,47.917072,75.79633,192.788882,0.0,0.0,0.0,0.0,0.0,2.613173,4.810663,17.993176,45.303899,70.985667,174.795706
43124559,1,"""below_25""","""donothing""","""below""",5818.486519,47.917072,75.79633,77.471365,0.0,10226.21739,0.0,0.0,0.0,2.613173,4.810663,5.707215,45.303899,70.985667,71.76415
43124559,2,"""above_25""","""donothing""","""above""",5855.612372,47.917072,75.79633,72.852551,0.0,11023.814965,0.0,0.0,0.0,2.613173,4.810663,7.901034,45.303899,70.985667,64.951517
43124559,3,"""even_25""","""donothing""","""even""",5839.033107,47.917072,75.79633,74.690363,0.0,10667.632393,0.0,0.0,0.0,2.613173,4.810663,6.970925,45.303899,70.985667,67.719438
43124559,4,"""clearcut_25""","""donothing""","""clearcut""",11688.695243,47.917072,75.79633,0.0423,0.0,17414.282066,0.0,0.0,0.0423,2.613173,4.810663,0.0,45.303899,70.985667,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
43124580,0,"""donothing""","""donothing""","""donothing""",8039.585346,121.824124,154.465368,285.34149,0.0,0.0,81.414022,98.505342,166.069155,10.689557,14.322556,28.068884,29.720546,41.63747,91.203451
43124580,1,"""below_25""","""donothing""","""below""",8509.550542,121.824124,154.465368,172.872991,0.0,10096.551727,81.414022,98.505342,99.016988,10.689557,14.322556,18.524791,29.720546,41.63747,55.331212
43124580,2,"""above_25""","""donothing""","""above""",8539.686933,121.824124,154.465368,170.355714,0.0,10743.99036,81.414022,98.505342,100.757673,10.689557,14.322556,15.418779,29.720546,41.63747,54.179262


In [ ]:
model = gp.Model()

hsum5 = 0
hsum25 = 0
npvsum = 0
stocksum = 0
mvars = {}
unitsums = defaultdict(float)
for row in df.iter_rows(named=True):
    new_var = model.addVar( vtype=GRB.BINARY )
    mvars[(row["unit"],row["treatment_5"],row["treatment_25"])] = new_var
    hsum5 += new_var*row["harvest_value_5"]
    hsum25 += new_var*row["harvest_value_25"]
    npvsum += new_var*row[f"npv_{discount_factor}_percent"]
    stocksum += new_var*row["stock_25"]
    unitsums[row["unit"]] += new_var

for unitsum in unitsums.values():
    model.addConstr(unitsum == 1)

harvest5 = model.addVar()
model.addConstr(harvest5 <= hsum5)
harvest25 = model.addVar()
model.addConstr(harvest25 <= hsum25)
stock = model.addVar()
model.addConstr(stock <= stocksum)
npv = model.addVar()
model.addConstr(npv <= npvsum)

model.setObjective(npv,sense=GRB.MAXIMIZE)
model.optimize()

print(model.ObjVal)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1245U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26 rows, 234 columns and 999 nonzeros
Model fingerprint: 0x50260403
Variable types: 4 continuous, 230 integer (230 binary)
Coefficient statistics:


  Matrix range     [2e-02, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
0.0
